## Reporting: wragle_report

### 1. Project Introduction

This project is focused on Data Wrangling. It focuses on gathering real-world data from Twitter and other sources captured in different formats, assessing the data quality and tidiness and then cleaning it to make it useful for further analysis.
There were three (3) datasets to be used. 
1. Twitter archive data of the user @dog_rates also known as WeRateDogs.(WeRateDogs is a twitter account that rates dogs with a humorous comment about the dog and give a rating.
2. Additional data to be gathered from the twitter API
3. Image predictions file available via a given link

### 2. Gathering Data

The twitter archive was provided for download in a csv format as "twitter_archive_enhanced.csv". This was read in pandas into a dataframe of the same name (twitter_archive_enhanced.csv).

The Image predictions file was downloaded programmatically by getting the url with the python's request library and writing the contents to a file. This file was also read into a dataframe as image_predictions.csv.

To get the addtional twitter data, a Twitter developer account was created and the necessary keys and codes were given to be able to gather data needed. This was done with the tweepy library to first create an API object and then loop through the tweet_id(unique identification for tweets) of the twitter_archive_enhanced table and then capture all other information surrounding each of the tweets. The gathered data was stored in a json file and then looped through to gather the retweet_count and favourite_count of each tweet in the twitter_archive_enhanced dataframe. The file was saved as twitter_api_data and read into a dataframe.

### 3. Assessing Data

Now that the datasets were ready and all read into dataframes, they had to be assessed. These were real-life data and they definitely needed some work done on them before proceeding. 9 Quality Issues and 4 Tidiness Issues were identified visually and programmatically.

Quality Issues
1. By displaying the characteristics of each dataset using the .info() method, it was realised that the id columns (tweet_id, retweet_id, etc were of the int64 datatype and had to be changed to string. ID's are mostly not used in computations.
2. The timestamp datatype in the twitter_archive_enhanced dataframe had to be changed to datetime to allow easy cleaning.
3. The image_prediction file only had predictions for tweets before 01/07/2017. The other tables had to be stripped of tweets gathered after this date.
4. Retweeted entries in the dataframes needed to be removed.
5. Similarly, tweet reply entries also needed to be removed.
6. The expanded_urls column of the twitter_archive_enhanced table had missing values and needed to be removed.
7. An quick inspection of the "name" column showed some weird dog_names. (Eg.Alphabets like "a", articles like "the",etc). These obviously were not real dog names.
8. Inconsistent ratings given. the usual WeRateDogs rating was a fraction mostly with a denominator of 10. The numerators were allowed to be slightly more than 10 simply because, "They're good dogs, Brent"
9. Some missing values were shown as "None".

Tidiness Issues
1. In gathering data from the twitter API, some additional characters were stored in the retweet and favourite count. These fields needed to be digits only.
2. Since the retweets and replies were to be deleted, the fields for retweet_id and reply_id were no longer needed.
3. The dog stage for some of the tweets were extracted and stored into separate columns. These had to be combined into 1 column.
4. The twitter_api_data had to be merged with the twitter_archive_enhanced table.

### 4. Cleaning Data

These are descriptions of the various methods used to clean the data:

Quality

1. The id columns were changed to str using the .astype() method in both the twitter_archive_enhanced & image_predictions table
2. .to_datetime() method was used to change the timestamp in the twitter_archive_enhanced dataframe
3. Now that the datetime had the right datatype, all tweets from 01/07/2017 were filtered and removed from the twitter_archive_enhanced dataframe.
4. All retweets and replies were removed by filtering out the entries with "retweeted_status_id" and deleting them.
5. All retweets and replies were removed by filtering out the entries with "in_reply_to_status_id" and deleting them.
6. The "expanded_urls" column with missing data were removed with .drop() and .isnul() methods.
7. Now this needed some tact. Erroneous dog names were extracted with a regex expression which returned all entries in the "name" column that begun with a small letter. (Based on the logic that the name of a dog is Proper Noun and will begin with a CAPITAL letter). These entries were then inspected again and the name of the dog if it existed in the tweet was extracted through another regex expression. (Most tweets had the phrase " This is my dog named .......")
8. The inconsistent ratings were cleaned by using regex expressions to search the tweet text for XX/XX format where the denominator was 10. Null values and Outliers were removed.
9. "None" entries were changed to numpy Null values.

Tidiness
1. Using the .strip() method, the retweet_count and favourite_count in the twitter_api_data dataframe were cleaned to have only count values.
2. retweet_status_id and in_reply_to_status_id were dropped with .drop() method.
3. To get all the dog stages for each entry stored in one column, I combined all dog stage entries to form a single string (Eg. doggo | floofer | pupper | puppa ==> doggoflooferpupperpuppa) and saved in a new column ("dog_stages"). The dog_stages were then extracted from this new string. For instances where two dog_stages were assigned to one tweet, it was analysed and the row was either dropped the relevant information pulled from the tweet text.
4. twitter_api_data was merged with the twitter_archive_enhanced table as they both had tweet_id in common. To do this, the twitter_api_data had to drop the first 57 entries as they corresponded to the tweets made from 01/07/2017. The index for both tables were then resetted and .merge() method used to combine both tables into a new table "twitter_archive_master".


### 5. Storing Data

The twitter_archive_master table was saved as a csv file programmatically .